In this first notebook, we will go from the installation of the [Brian](http://briansimulator.org/) neural simulator to the display of the simulation of an Hodgkin-Huxley neuron.

# Installation of brian

In [ ]:
!python3 -m pip install -U brian2

We will now explore some examples of their [tutorials](https://brian2.readthedocs.io/en/stable/resources/tutorials/index.html)

In [ ]:
import brian2 as sim

In [ ]:
help(sim.core)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
fig_width = 15
phi = (np.sqrt(5)+1)/2

# the simplest neuron in the world


## Units system

Brian has a system for using quantities with physical dimensions:

In [ ]:
20*sim.volt

All of the basic SI units can be used (volt, amp, etc.) along with all the standard prefixes (m=milli, p=pico, etc.), as well as a few special abbreviations like ``mV`` for millivolt, ``pF`` for picofarad, etc.

In [ ]:
1000*sim.amp

In [ ]:
1e6*sim.volt

In [ ]:
1000*sim.namp

Also note that combinations of units with work as expected:

In [ ]:
10*sim.nA*5*sim.Mohm

## A simple model

Let's start by defining a simple neuron model. 

In [ ]:
sim.start_scope()

tau = 10*sim.ms
v0 = 42*sim.volt
eqs = '''
dv/dt = (v0-v)/tau : volt
'''

G = sim.NeuronGroup(1, eqs)
M = sim.StateMonitor(G, 'v', record=True)

sim.run(100*sim.ms)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(M.t/sim.ms, M.v[0])
ax.set_xlabel('Time (ms)')
ax.set_ylabel('v (volts)');

## Adding spikes

So far we haven't done anything neuronal, just played around with differential equations. Now let's start adding spiking behaviour.

In [ ]:
sim.start_scope()

tau = 10*sim.ms
v0 = 1.2*sim.volt
eqs = '''
dv/dt = (v0-v)/tau : volt
'''

G = sim.NeuronGroup(1, eqs, threshold='v>0.8*volt', reset='v=0*volt', refractory=5*sim.ms)
M = sim.StateMonitor(G, 'v', record=True)
spikemon = sim.SpikeMonitor(G)

sim.run(100*sim.ms)


In [ ]:
print('Spike times: %s' % spikemon.t[:])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(M.t/sim.ms, M.v[0])
for t in spikemon.t:
    ax.vlines(t/sim.ms, ymin=.8, ymax=1.2)
ax.set_xlabel('Time (ms)')
ax.set_ylim(-.2, 1.4)
ax.set_ylabel('v');

### Hodgkin-Huxley cells (without morphology)

code adapted from https://brian2.readthedocs.io/en/stable/examples/IF_curve_Hodgkin_Huxley.html  (morphology added in  https://brian2.readthedocs.io/en/stable/examples/compartmental.hodgkin_huxley_1952.html ).

In [ ]:
sim.start_scope()

duration = .2*sim.second

# Parameters
area = 20000*sim.umetre**2
Cm = 1*sim.ufarad*sim.cm**-2 * area
gl = 5e-5*sim.siemens*sim.cm**-2 * area
El = -65*sim.mV
EK = -90*sim.mV
ENa = 50*sim.mV
g_na = 100*sim.msiemens*sim.cm**-2 * area
g_kd = 30*sim.msiemens*sim.cm**-2 * area
VT = -63*sim.mV

# The model
HH_eqs = sim.Equations('''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
    (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
    (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
    (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
''')
# Threshold and refractoriness are only used for spike counting
HH_G = sim.NeuronGroup(1, HH_eqs,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
HH_G.v = El
HH_G.I = '0.7*nA'

HH_M = sim.StateMonitor(HH_G, 'v', record=True)

sim.run(duration)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/phi))
ax.plot(HH_M.t/sim.ms, HH_M.v[0])
ax.set_xlabel('Time (ms)')
ax.set_ylim(-.1, .07)
ax.set_ylabel('v (volt)');